In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, RidgeCV, Lasso

In [10]:
df=pd.read_csv('/rds/general/user/hsl121/home/hda_project/rq1/rq1_cleaned_no_ae.csv')
adverse=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Adverse Events 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')

In [15]:

adverse.columns
print(adverse['Withdrawal'].isna().sum())
print(adverse['Withdrawal'].unique())
## One Hot Encoding for Adverse Events (just predictin yes or no adverse events)

def adverse_na(x):
    if pd.isna(x):
        return 'No'
    else:
        return x
def adverse_ohe(x):
    if x=='No':
        return 0
    else:
        return 1

adverse_cols= [col for col in adverse.columns
               if col not in ['SID', 'Prescribed', 'BaselinePROMS', 'Date', 'DaysFromStart']
               and not col.endswith('days')]


adverse[adverse_cols] = adverse[adverse_cols].applymap(adverse_na)
adverse[adverse_cols] = adverse[adverse_cols].applymap(adverse_ohe)



rename_dict = {col: f"{col}_adverse" for col in adverse_cols}

# Rename the columns in the adverse subset
selected_adverse = adverse[['SID'] + adverse_cols].rename(columns=rename_dict)
selected_adverse.shape
selected_adverse = selected_adverse.groupby('SID', as_index=False).max()


full_adverse = pd.merge(df, selected_adverse, on='SID', how='left')
full_adverse.to_csv("/rds/general/user/hsl121/home/hda_project/rq1/rq1_cleaned_yes_ae.csv", index=False) 

0
[1]


/rds/general/ephemeral/user/hsl121/ephemeral/ipykernel_563890/3451916867.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  adverse[adverse_cols] = adverse[adverse_cols].applymap(adverse_na)
/rds/general/ephemeral/user/hsl121/ephemeral/ipykernel_563890/3451916867.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  adverse[adverse_cols] = adverse[adverse_cols].applymap(adverse_ohe)


In [16]:
full_adverse.shape

(20593, 266)

In [17]:
# create binary col - if any of the adverse cols=1, then binary col =1, else =0
adverse_cols_renamed = [f"{col}_adverse" for col in adverse_cols]
full_adverse['adverse_binary'] = full_adverse[adverse_cols_renamed].sum(axis=1).gt(0).astype(int)
full_adverse = full_adverse.drop(columns=adverse_cols_renamed, axis=1)
full_adverse.to_csv("/rds/general/user/hsl121/home/hda_project/rq1/rq1_cleaned_adverse_binary.csv", index=False) 

Binary AE Added

In [18]:
df=pd.read_csv('/rds/general/user/hsl121/home/hda_project/rq1/rq1_cleaned_adverse_binary.csv')

In [20]:
df.columns.tolist()

['SID',
 'EQ5D_Round1',
 'EQ5D_Round2',
 'EQ5D_Round3',
 'EQ5D_Round4',
 'EQ5D_Round5',
 'EQ5D_Round6',
 'Age',
 'Sex',
 'occupation',
 'weight',
 'height',
 'Myocardial_infarction',
 'Congestive_heart_failure',
 'Peripheral_vascular_disease',
 'Cerebrovascular_accident_or_transient_ischemic_attack',
 'Dementia',
 'Chronic_obstructive_pulmonary_disease',
 'Connective_tissue_disease',
 'Peptic_Ulcer_Disease',
 'Liver_disease',
 'Diabetes',
 'Hemiplegia',
 'Moderate_to_severe_chronic_kidney_disease',
 'Solid_tumour',
 'Leukemia',
 'Lymphoma',
 'AIDS',
 'Charlson_comorbidity',
 'Hypertension',
 'Depression_or_anxiety',
 'Arthritis',
 'Epilepsy',
 'VTE',
 'Endocrine_thyroid_dysfunction',
 'Allergy',
 'Smoking_status',
 'Smoking_pack_years',
 'alcohol_units',
 'Cannabis_status',
 'GAD7_Round1',
 'insomniaEfficacyMeasure_Round1',
 'diag_Depression',
 'diag_Anxiety',
 'diag_Chronic pain',
 'diag_Osteoarthritis',
 'diag_PTSD',
 'diag_Fibromyalgia',
 'diag_Multiple sclerosis',
 'diag_Neuropathi

In [4]:
df.columns
drop=['EQ5D_Round2', 'EQ5D_Round3', 'EQ5D_Round4','EQ5D_Round5', 'EQ5D_Round6']
df=df.drop(columns=drop)

In [5]:
full=df.copy()

In [6]:
X=full.drop(['adverse_binary','SID'], axis=1)
y=full['adverse_binary']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [39]:
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.3f}")
print(f"Test R²: {r2:.3f}")


Test MSE: 0.052
Test R²: 0.022


XGBoost

In [8]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    scale_pos_weight=(y == 0).sum() / (y == 1).sum()  # handle imbalance
)

xgb_model.fit(X_train, y_train)


/rds/general/user/hsl121/home/anaconda3/envs/TDS/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [20:22:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
y_pred = xgb_model.predict(X_test)
y_proba = xgb_model.predict_proba(X_test)[:, 1]

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))


Confusion Matrix:
 [[3572  316]
 [ 185   46]]

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      3888
           1       0.13      0.20      0.16       231

    accuracy                           0.88      4119
   macro avg       0.54      0.56      0.54      4119
weighted avg       0.90      0.88      0.89      4119

ROC AUC Score: 0.662260835871947
